# Persisting Layer for Batch Job

** Importing Requisites **

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext

import time

from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

In [18]:
# Creating Spark Session and SQL Context
spark = SparkSession.builder.appName('lr_example').getOrCreate()
sqlContext = SQLContext(spark)

** Reading Data from HDFS **

In [19]:
data = spark.read.csv("hdfs://localhost:9000/kafka_spark", inferSchema=True, header=True)

In [20]:
data.printSchema()

root
 |-- action: string (nullable = true)
 |-- page: string (nullable = true)
 |-- prevPage: string (nullable = true)
 |-- product: string (nullable = true)
 |-- referrer: string (nullable = true)
 |-- timestamp_hour: double (nullable = true)
 |-- visitor: string (nullable = true)



In [21]:
data.columns

['action',
 'page',
 'prevPage',
 'product',
 'referrer',
 'timestamp_hour',
 'visitor']

In [22]:
data.show()

+-----------+-------+--------+--------------------+--------+-----------------+--------------+
|     action|   page|prevPage|             product|referrer|   timestamp_hour|       visitor|
+-----------+-------+--------+--------------------+--------+-----------------+--------------+
|   purchase| Page-3|  Page-2|David's Tea,Choco...|Internal|1.522059079456E12|Visitor-200957|
|add_to_cart| Page-8|    null|Motorola,SURFboar...|    Bing|1.522059079936E12|Visitor-142811|
|  page_view|Page-10|    null|L'Oreal,Out of Be...| Twitter|1.522059079936E12|Visitor-900950|
|  page_view| Page-2|    null|      Coca Cola,Soda|  Direct|1.522059079936E12|Visitor-878781|
|  page_view| Page-8|    null|Scotch,Super 33+ ...|   Yahoo|1.522059080416E12|Visitor-963300|
|  page_view| Page-1|    null|Simple,Eye Makeup...|  Google|1.522059080416E12|Visitor-332696|
|  page_view| Page-2|    null|Fernet-Branca,Liq...|Facebook|1.522059080416E12|Visitor-483822|
|  page_view| Page-5|    null|Reynolds,Slow Coo...|Facebook|

In [23]:
data.show(7, truncate=False)

+-----------+-------+--------+-----------------------------------------------------+--------+-----------------+--------------+
|action     |page   |prevPage|product                                              |referrer|timestamp_hour   |visitor       |
+-----------+-------+--------+-----------------------------------------------------+--------+-----------------+--------------+
|purchase   |Page-3 |Page-2  |David's Tea,Chocolate Chili Chai Black Tea           |Internal|1.522059079456E12|Visitor-200957|
|add_to_cart|Page-8 |null    |Motorola,SURFboard eXtreme Cable Modem (Model SB6121)|Bing    |1.522059079936E12|Visitor-142811|
|page_view  |Page-10|null    |L'Oreal,Out of Bed Texturizer                        |Twitter |1.522059079936E12|Visitor-900950|
|page_view  |Page-2 |null    |Coca Cola,Soda                                       |Direct  |1.522059079936E12|Visitor-878781|
|page_view  |Page-8 |null    |Scotch,Super 33+ Vinyl Electric Tape                 |Yahoo   |1.522059080416E12|

In [24]:
def timestamp_conversion(timestamp):
        if timestamp is not None:
            return time.strftime('%Y-%m-%d', time.gmtime(timestamp/1000))
        else:
            return "null null"

In [25]:
time_udf = udf(timestamp_conversion,StringType())

In [26]:
inputDF = data.withColumn("timestamp_hour",time_udf(data["timestamp_hour"]))
inputDF.createOrReplaceTempView("activity")

** Batch Analytics **

In [27]:
visitorsByProduct = sqlContext.sql("SELECT product, timestamp_hour, COUNT(DISTINCT visitor) as unique_visitors FROM Activity GROUP BY product, timestamp_hour")

In [28]:
visitorsByProduct.show(5)

+--------------------+--------------+---------------+
|             product|timestamp_hour|unique_visitors|
+--------------------+--------------+---------------+
|Ziploc,Double Zip...|    2018-03-26|            411|
|Clif Bar,Chocolat...|    1970-01-01|            214|
|Scotch,Super 33+ ...|    1970-01-01|            212|
|  Kroger,Garlic Salt|    1970-01-01|            232|
|Trader Joe's,Toma...|    2018-03-26|            439|
+--------------------+--------------+---------------+
only showing top 5 rows



In [29]:
activityByProduct = sqlContext.sql("SELECT product, timestamp_hour, sum(case when action = 'purchase' then 1 else 0 end) as purchase_count, sum(case when action = 'add_to_cart' then 1 else 0 end) as add_to_cart_count, sum(case when action = 'page_view' then 1 else 0 end) as page_view_count from Activity group by product, timestamp_hour").cache()

In [30]:
activityByProduct.show(5)

+--------------------+--------------+--------------+-----------------+---------------+
|             product|timestamp_hour|purchase_count|add_to_cart_count|page_view_count|
+--------------------+--------------+--------------+-----------------+---------------+
|Ziploc,Double Zip...|    2018-03-26|             9|               16|            386|
|Clif Bar,Chocolat...|    1970-01-01|            12|                8|            194|
|Scotch,Super 33+ ...|    1970-01-01|             6|                6|            200|
|Trader Joe's,Toma...|    2018-03-26|            13|                7|            419|
|  Kroger,Garlic Salt|    1970-01-01|             5|               10|            217|
+--------------------+--------------+--------------+-----------------+---------------+
only showing top 5 rows



** Creating Cassandra cluster object and connecting to default cluster **

In [31]:
cluster = Cluster()
session = cluster.connect()

# Setting keyspace so that all queries by default refer to this keyspace
session.set_keyspace('lambda')

** inserting into batch_visitors_by_product table in cassandra **

In [ ]:
query = session.prepare("INSERT INTO batch_visitors_by_product (product, timestamp_hour, unique_visitors) VALUES (?,?,?)")
for row in visitorsByProduct.rdd.collect():
    session.execute(query, [row['product'], row['timestamp_hour'], row['unique_visitors']])

** inserting into batch_activity_by_product ** 

In [ ]:
query = session.prepare("INSERT INTO batch_activity_by_product (product, timestamp_hour, add_to_cart_count, page_view_count, purchase_count) VALUES (?, ?, ?, ?, ?)")
for row in activityByProduct.rdd.collect():
    session.execute(query, [row['product'], row['timestamp_hour'], row['purchase_count'], row['add_to_cart_count'],row['page_view_count']])

** D O N E ! . . . . . **